In [2]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [1]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/mlm', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/modules']


2024-06-04 04:35:32.945612: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-04 04:35:33.172134: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 04:35:33.953564: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from datasets import load_from_disk,concatenate_datasets


adapter_name="mlm_union_TR"
dataset = load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/datasets")



In [4]:
reload(config)

    
filtered_target = dataset['train'].filter(lambda example: example['genre'] == "travel").shuffle(seed=42)
#filtered_target = shuffled_filtered_target.train_test_split(test_size=0.1)


filtered_test_target = dataset['validation_matched'].filter(lambda example: example['genre'] == "travel")
train_target = filtered_target
test_target = filtered_test_target

train_target[1]

{'promptID': 82447,
 'pairID': '82447e',
 'premise': 'Wherever the original furnishings and decoration were missing, superb appropriate equivalents have been installed.',
 'premise_binary_parse': '( ( Wherever ( ( the ( original ( ( furnishings and ) decoration ) ) ) ( were missing ) ) ) ( , ( ( ( superb appropriate ) equivalents ) ( ( have ( been installed ) ) . ) ) ) )',
 'premise_parse': '(ROOT (S (S (VP (VB Wherever) (SBAR (S (NP (DT the) (JJ original) (NNS furnishings) (CC and) (NN decoration)) (VP (VBD were) (ADJP (VBG missing))))))) (, ,) (NP (ADJP (JJ superb) (JJ appropriate)) (NNS equivalents)) (VP (VBP have) (VP (VBN been) (VP (VBN installed)))) (. .)))',
 'hypothesis': 'Superb equivalents have been installed in the place of the missing original furnishings.',
 'hypothesis_binary_parse': '( ( Superb equivalents ) ( ( have ( been ( installed ( in ( ( the place ) ( of ( the ( missing ( original furnishings ) ) ) ) ) ) ) ) ) . ) )',
 'hypothesis_parse': '(ROOT (S (NP (NNP Superb

In [5]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
init(model)
reload(fn)
fn.print_trainable_parameters(model)

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

trainable params: 109514298 || all params: 109514298 || trainable%: 100.0


In [6]:
from adapters import ConfigUnion, LoRAConfig, PrefixTuningConfig, SeqBnConfig,SeqBnInvConfig,AdapterConfig,LoRAConfig

union_config = ConfigUnion(
    SeqBnInvConfig(reduction_factor=2),
    LoRAConfig(r=8, alpha=16),
)
model.add_adapter(adapter_name, config=union_config)

model.train_adapter([adapter_name])
model.active_adapters = adapter_name
print(model.adapter_summary())
fn.print_trainable_parameters(model)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
mlm_union_TR             union             7,682,688       7.055       1       1
--------------------------------------------------------------------------------
Full model                               108,891,648     100.000               0
trainable params: 8274816 || all params: 117196986 || trainable%: 7.060604783812444


In [7]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(train_target,tokenizer)
tokenized_dataset

Map:   0%|          | 0/77350 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 77350
})

In [8]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset[444:470]

results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [9]:
# we group texts and chunk them
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Map:   0%|          | 0/77350 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 27252
})

In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] as you reach [MASK] atlantic coast, you'll become almost engulfed in vast groves of banana trees, with bunches hanging temptingly close to [MASK] [MASK]. [SEP] vast groves of banana trees will almost engulf you. [SEP] [CLS] wherever [MASK] original furnishings [MASK] decoration were missing [MASK] superb appropriate equivalent [MASK] have been installed. [SEP] superb [MASK]s have been installed in the place [MASK] the missing original furnishingsе [SEP] [CLS] in a low - rise building near the cave entrance is a small ethnographic [MASK], a place to [MASK] displays of [MASK] island customs and costumes as you wait a couple of minutes for the guided cave tour [MASK] beginral [SEP] there is'

'>>> a small museum near the cave [MASK]rting [SEP] [CLS] by allowing the local pride of such historic regions as provence, normandy, brittany, and [MASK]ue? doc romanian [MASK]assert itself, [unused144] [MASK] that it was at last secure inmbe national identity [MASK] so secure that frenc

In [11]:

downsampled_dataset = lm_datasets.train_test_split(
    test_size=0.1, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 24526
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2726
    })
})

In [12]:

reload(fn)
trainer = fn.train_mlm_model(model,adapter_name,data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['test'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [13]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/86 [00:00<?, ?it/s]

>>> Perplexity: 13.57


In [14]:
trainer.train()

  0%|          | 0/15340 [00:00<?, ?it/s]

{'loss': 2.1748, 'grad_norm': 1.835453987121582, 'learning_rate': 9.820754716981132e-05, 'epoch': 1.0}


  0%|          | 0/86 [00:00<?, ?it/s]

{'eval_loss': 1.8411283493041992, 'eval_runtime': 4.6962, 'eval_samples_per_second': 580.466, 'eval_steps_per_second': 18.313, 'epoch': 1.0}
{'loss': 1.9331, 'grad_norm': 1.760197401046753, 'learning_rate': 9.304582210242589e-05, 'epoch': 2.0}


  0%|          | 0/86 [00:00<?, ?it/s]

...

  0%|          | 0/86 [00:00<?, ?it/s]

{'eval_loss': 1.5418118238449097, 'eval_runtime': 4.9171, 'eval_samples_per_second': 554.396, 'eval_steps_per_second': 17.49, 'epoch': 18.0}


Overwriting existing adapter 'mlm_union_TR'.


{'train_runtime': 1776.2269, 'train_samples_per_second': 276.158, 'train_steps_per_second': 8.636, 'train_loss': 1.7347791625609628, 'epoch': 18.0}


TrainOutput(global_step=13806, training_loss=1.7347791625609628, metrics={'train_runtime': 1776.2269, 'train_samples_per_second': 276.158, 'train_steps_per_second': 8.636, 'train_loss': 1.7347791625609628, 'epoch': 18.0})

In [15]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/86 [00:00<?, ?it/s]

>>> Perplexity: 4.80


In [16]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)


: 

#### This is a report of mlm for TRAVEL domain using whole genre as target and then splitt after tokenization with seed 42. I used computer metrix for preplexity and early stopping 
The results before:
>>> Perplexity: 13.57

The results after:
>>> Perplexity: 4.80